# Project Background

## Objective
For those who are looking for data science jobs, it is important to know where those jobs are located. The goal of this work is to identify metropolitan areas in the United States with high counts of data science jobs and those seeing growth, using geographic mapping, employment trend analysis, and cost of living analysis.

## Data Sources
This work has three main datasources. 
1. **U.S. Census Bureau**: Provides geographic data (shapefiles) to map metropolitan areas across the country: https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2024&layergroup=Core+Based+Statistical+Areas
2. **Bureau of Labor Statistics** -Occupational Employment and Wage Statistics: Offers employment counts and wages by metropolitan area: https://www.bls.gov/oes/tables.htm
3. **Bureau of Economic Analysis**: Supplies a cost-of-living index known as the Regional Price Parities (table-MARPP Real personal income and regional price parities by MSA): https://apps.bea.gov/itable/?ReqID=70&step=1&_gl=1

## Limitations
1. Employment data is based on an annual estimate from May 2024 and does not reflect real-time job changes.
2. The analysis focuses narrowly on the "data scientist" role, excluding other data-related positions such as data analysts and statisticians.
3. Key limitations to overtime analysis include: 
    * Changes in SOC Codes: The Standard Occupational Classification (SOC) codes may evolve, impacting the comparability of job definitions across years. The most recent updates were in 2018.
    * Methodological Changes: Employment estimation methodologies have changed, with the MB3 methodology introduced in 2021. Comparisons should only be made between years using the same methodology.
    * Merging of MSAs: In 2024, some metropolitan statistical area (MSA) definitions were redefined, particularly in the northeastern U.S. This could affect historical comparisons. To learn more about these changes, visit https://www.bls.gov/oes/notices/.

# Key Findings

1. **Top Metros by Data Science Employment (2024)**
* As of 2024, the metros with the highest number of data science employees are as follows: 
    * New York-Newark-Jersey City, NY-NJ – 19,810
    * Los Angeles-Long Beach-Anaheim, CA – 10,030
    * San Francisco-Oakland-Fremont, CA – 9,350
    * Dallas-Fort Worth-Arlington, TX – 8,630
    * Washington-Arlington-Alexandria, DC-VA-MD-WV – 8,530

Check out this interactive metro-map: https://abeanhz.github.io/Projects/EconomicAnalysis/DataScienceEmployment/map_employee_count.html

2. **Comparing Employment Concentration Across Metros**
* While metros like New York and Los Angeles have the highest overall counts of data scientists, some smaller areas have a much higher concentration of data scientists relative to the national average. This is captured using the location quotient (LQ), which compares the proportion of data scientists in a metro to the national proportion.
    * In the San Jose-Sunnyvale-Santa Clara, CA metro, data scientists are nearly 4 times more common than the U.S. average (LQ = 3.8).
    * In Durham-Chapel Hill, NC, data scientists are almost 3 times more common than the national average (LQ = 2.98).
    * In Bloomington, IL, data scientists are also about 3 times more common than average (LQ = 2.85).

Check out this interactive metro-map: https://abeanhz.github.io/Projects/EconomicAnalysis/DataScienceEmployment/map_LQ.html

Look at who is the in top 25 for employees count vs. employment concentration: https://abeanhz.github.io/Projects/EconomicAnalysis/DataScienceEmployment/top25_comparison_table.html

3. **Fastest-Growing Data Science Hubs with at Least 1000 Employees in 2024** 
* Among the top 25 metros (by 2024 employment counts), several Texas cities have experienced rapid growth in data science employment over the past three years:
    * Houston-Pasadena-The Woodlands, TX: +441% (2021-2023)
    * Austin-Round Rock-San Marcos, TX: +204% (2021-2023)
    * San Antonio-New Braunfels, TX: +176% (2021-2023)
    * Dallas-Fort Worth-Arlington, TX: +150% (2021-2023)
    
Check out this interactive table that shows the 2-year percent growth (2021-23) in employees: https://abeanhz.github.io/Projects/EconomicAnalysis/DataScienceEmployment/Growth_table_interactive.html

4. **Balancing Salary and Cost of Living**
* Using the Regional Price Parity (RPP) Index alongside median salaries provides insight into affordability for data scientists.
    * Charlotte-Concord-Gastonia, NC-SC and Bloomington, IL stand out with a lower-than-average cost of living and a higher-than-national median data scientist salary, making it a potentially ideal metro to work in due to competitive salaries.
    * In contrast, some metros with a high cost of living report salaries below the national median, which may reduce relative earning power.

Explore these trends here: https://abeanhz.github.io/Projects/EconomicAnalysis/DataScienceEmployment/top25Comparison_costofliving_Median.html

5. **Top 3 industries employing data scientists**
* The three industries employing the largest number of data scientists are:


    * Professional, Scientific, and Technical Services
    * Management of Companies and Enterprises
    * Insurance Carriers and Related Activities

# Top Industries in 2024

In [49]:
#!pip install geopandas shapely plotly
#!pip install thefuzz
#!pip install --upgrade nbformat


In [50]:
import pandas as pd  ### Basic data manipulations
import matplotlib.pyplot as plt
import os
import geopandas as gpd ### using shapefiles

import plotly.express as px ### choropleth map
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import json ### to read the converted shapefiles
from thefuzz import process ### for merging on MSA with slightly different labeling patterns
import numpy as np  ### dealing with missing data and other calcs

import warnings
warnings.filterwarnings("ignore")  ### warnings sometimes show your computer, so I want to surpress if sending to github


Need to filter and sort the tables to find highest employment by industry.

In [51]:
industry_4digit_df = pd.read_excel('Data\\nat4d_M2024_dl.xlsx', sheet_name="Nat4d_M2024_dl") ### read in national employment by industry
industry_3digit_df = pd.read_excel('Data\\nat3d_M2024_dl.xlsx', sheet_name="nat3d_M2024_dl")

In [52]:
def create_industry_rank(df): 
    create_df = df[df['OCC_CODE']=='15-2051'] ### # Filter data data science jobs 
    create_df = create_df[create_df['OWN_CODE']==5] ### filter to private industry
    create_df["TOT_EMP"] = pd.to_numeric(create_df["TOT_EMP"], errors="coerce") ### Convert to numeric - some * for anonymized
    create_df = create_df.sort_values(by="TOT_EMP", ascending=False) ### Sort by employment
    return(create_df)

In [53]:
# Tables with employment counts by NAICS industry codes
nation_datascience4 = create_industry_rank(industry_4digit_df)
nation_datascience3 = create_industry_rank(industry_3digit_df)

# Mapping Data Scientist Job Hotspots in the U.S.

For this analysis, I am going to explore which metropolitan areas had the highest number of data science jobs across the United States. 

Data Sources: 
* Occupational Employment and Wage Statistics (OEWS) Tables https://www.bls.gov/oes/tables.htm 
* Shapefiles: https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2024&layergroup=Core+Based+Statistical+Areas

## Load Data

In [54]:
# Load the shapefile
shapefile_path = "CensusShapeFiles/tl_2024_us_cbsa.shp"
gdf = gpd.read_file(shapefile_path)
census_locale =gdf[['NAME']]

In [55]:
# Load OEWS Data -- this step will take awhile to read-in the full df
Employment = pd.read_excel('Data\\MSA_M2024_dl.xlsx', sheet_name="MSA_M2024_dl")  ### read in the data
OEWS_locale = Employment[['AREA_TITLE']].drop_duplicates()

## Merge data

There are subtle differences in some of the naming conventions for the MSAs in the census and the OEWS data so we have to do some matching to make a map between the two data sources. 

Merge 1 --  Exact matches

In [56]:
# Keeping exact matches (based on the data in OEWS df)
exact_match = pd.merge(census_locale, OEWS_locale, left_on='NAME', right_on='AREA_TITLE', how='inner')

In [57]:
# Checking un-matched geographies
anti_join = pd.merge(census_locale, OEWS_locale, left_on='NAME', right_on='AREA_TITLE', how='right')
anti_join = anti_join[anti_join['NAME'].isna()] ### keep only NA values to find what BLS values did not have a geo match
anti_join = anti_join.dropna(axis=1, how='all') ### dropping cols filled with NAs to try a fuzzy match on remaining missing locations


Merge 2 -- Fuzzy Matches

In [58]:
# Function to find best match
def fuzzy_merge(census_locale, anti_join, left_col, right_col, threshold=88):
    matches = census_locale[left_col].apply(lambda x: process.extractOne(x, anti_join[right_col], score_cutoff=threshold))
    census_locale['Best_Match'] = [match[0] if match else None for match in matches]
    return pd.merge(census_locale, anti_join, left_on='Best_Match', right_on=right_col, how='right').drop(columns=['Best_Match'])

# Perform Fuzzy Merge
fuzzy_df = fuzzy_merge(census_locale, anti_join, 'NAME', 'AREA_TITLE')
fuzzy_df.head() ### manually check that the fuzzy produced logical matches


,NAME,AREA_TITLE
0,"Mayagüez, PR","Mayaguez, PR"
1,"San Juan-Bayamón-Caguas, PR","San Juan-Bayamon-Caguas, PR"


In [59]:
# Combine MSA Maps into a final map
MSA_MAP = pd.concat([exact_match, fuzzy_df], ignore_index=True)

## Prep data tables

Let's read in the BLS employment by occupation data and filter to just data science jobs. Some of the data is masked due to small employment counts or concerns over de-identification; these observations will be filtered out of the analysis.

In [60]:
def create_MSAEmploy(df): 
    #  OEWS Data
    create_employ = df[df['OCC_CODE']=='15-2051'] ### Filter to just data science occupations
    create_employ ["TOT_EMP"] = pd.to_numeric(create_employ["TOT_EMP"], errors="coerce") ### Convert to numeric 
    create_employ = create_employ.dropna(subset=["TOT_EMP"])  ### Remove rows where TOT_EMP is not reported
    create_employ = create_employ.sort_values(by="TOT_EMP", ascending=False) ### Sort greatest to least
    create_employ["A_MEDIAN"] = pd.to_numeric(create_employ["A_MEDIAN"], errors="coerce") ### Convert to numeric 
    return(create_employ)

MSA_compare = create_MSAEmploy(Employment)

In [61]:
# Use the ID map to create a column to be able to merge the employment data with the shapefile 
MSA_employ = pd.merge(MSA_compare, MSA_MAP, left_on='AREA_TITLE', right_on='AREA_TITLE', how='inner')

Now let's read in the shape files from the census bureau. Since the shapefiles are geom-shapes and we want to make a symbol map, we'll need to compute a centroid for each MSA in order to pin-point the area on the map.

In [62]:
# Extract MSA centroids
gdf["centroid"] = gdf.geometry.centroid
gdf["lon"] = gdf["centroid"].x
gdf["lat"] = gdf["centroid"].y


In [63]:
# Combine the shapefile with the employment data
geo_employ = pd.merge(gdf, MSA_employ, left_on="NAME", right_on="NAME", how = 'right') ### want a right join because only want metros w/ employment data

## Create Map

In [96]:
# Bubble size 
if "TOT_EMP" in geo_employ.columns:
    geo_employ["Employee Count"] = geo_employ["TOT_EMP"]
else:
    geo_employ["Employee Count"] = 10  # Default size if no population data

# Create a DataFrame for Plotly
figure_df = geo_employ[["lon", "lat", "NAME", "Employee Count", "A_MEDIAN"]] 

# Set up the mapbox layout
fig = px.scatter_mapbox(figure_df, 
                        lon="lon", 
                        lat="lat", 
                        size="Employee Count",  # Bubble size based on employment count
                        color="Employee Count",  # Color bubbles based on employment count
                        hover_name="NAME",  # Name to show on hover
                        size_max=35,  # Max bubble size
                        color_continuous_scale="Plasma",  # Color scale for employment count
                        title="Data Scientists <br><sup>2024 Employee Count, by Metropolitan Area")

# Update the map style and layout
fig.update_layout(
    mapbox_style="carto-positron",  
    mapbox_zoom=3,  # Zoom level for the map
    mapbox_center={"lat": 37.0902, "lon": -95.7129},  # Center map on the U.S., adjust this as needed
    showlegend=True,  # Ensure the legend is shown
    annotations=[
        dict(
            text="Source: Bureau of Labor Statistics - OEWS",
            xref="paper", yref="paper",
            x=0, y=-0.10,
            showarrow=False,
            align="left",  # <-- aligns text to the left
            font=dict(size=10, color="gray")
        )
    ]
)


fig.update_traces(
    hovertemplate="<b>%{hovertext}</b><br>Employee Count: %{marker.size:,}<extra></extra>"
)

# Show the map
fig.show()

In [65]:
fig.write_html("map_employee_count.html")

# How common are data science jobs? 

The location quotient is a metric that explains if a job is more or less common in a specific area compared to the national average.
* An LQ of 1.0 means the job is exactly as common in that area as it is nationally.
* An LQ greater than 1.0 means the job is more concentrated in that area.
* An LQ less than 1.0 means the job is less concentrated in that area.

For example, if LQ = 2, then data scientists are twice as common in this metro as they are in the U.S. overall.
If LQ = 0.5, then data scientists are half as common there as they are in the U.S. overall.

For more information, check out this documentation: https://www.bls.gov/cew/about-data/location-quotients-explained.htm

In [66]:
def create_LQ(df): 
    #  OEWS Data
    create_employ = df[df['OCC_CODE']=='15-2051'] ### Filter to just data science occupations
    create_employ["LOC_QUOTIENT"] = pd.to_numeric(create_employ["LOC_QUOTIENT"], errors="coerce") ### Convert to numeric 
    create_employ = create_employ.dropna(subset=["LOC_QUOTIENT"])  ### Remove rows where LOC_QUOTIENT is not reported
    create_employ = create_employ.sort_values(by="LOC_QUOTIENT", ascending=False) ### Sort greatest to least
    return(create_employ)

LQ_df = create_LQ(Employment)

In [67]:
# Use the ID map to create a column to be able to merge the employment data with the shapefile 
LQ_df = pd.merge(LQ_df, MSA_MAP, left_on='AREA_TITLE', right_on='AREA_TITLE', how='inner')

# Combine the shapefile with the employment data
geo_LQ = pd.merge(gdf, LQ_df, left_on="NAME", right_on="NAME", how = 'right') ### want a right join because only want metros w/ LQ data

In [ ]:
# Bubble size 
if "LOC_QUOTIENT" in geo_LQ.columns:
    geo_LQ["Concentration"] = geo_LQ["LOC_QUOTIENT"]
else:
    geo_LQ["Concentration"] = 0  # Default size if no population data

# Create a DataFrame for Plotly
fig_LQ_df = geo_LQ[["lon", "lat", "NAME", "Concentration", 'TOT_EMP']]  


# Set up the mapbox layout
LQ_map = px.scatter_mapbox(fig_LQ_df , 
                        lon="lon", 
                        lat="lat", 
                        size="Concentration",  
                        color="Concentration", 
                        hover_name="NAME",  # Name to show on hover
                        size_max=20,  # Max bubble size
                        color_continuous_scale="Viridis", 
                        title="Data Scientists <br><sup>2024 Employment Concentration, by Metropolitan Area")

# Update the map style and layout
LQ_map.update_layout(
    mapbox_style="carto-positron",  
    mapbox_zoom=3,  # Zoom level for the map
    mapbox_center={"lat": 37.0902, "lon": -95.7129},  # Center map on the U.S., adjust this as needed
    showlegend=True,  # Ensure the legend is shown
    annotations=[
        dict(
            text="Source: Bureau of Labor Statistics - OEWS",
            xref="paper", yref="paper",
            x=0, y=-0.10,
            showarrow=False,
            align="left",  # <-- aligns text to the left
            font=dict(size=10, color="gray")
        )
    ]
)

LQ_map.update_traces(
    hovertemplate="<b>%{hovertext}</b><br>Concentration: %{marker.size:,}<extra></extra>"
)


In [ ]:
formatted_employment = [f"{x:,.0f}" for x in fig_LQ_df["TOT_EMP"]]

LQ_table = go.Figure(data=[go.Table(
    header=dict(values=['Metropolitan Area', 'Concentration', 'Total Employment'],
                fill_color='#003900',
                align='left',
                font_color='white'),
    cells=dict(values=[
        fig_LQ_df["NAME"],
        fig_LQ_df["Concentration"],
        formatted_employment
    ],
    fill_color='#C1E1C1',
    align='left'))
])

LQ_table.update_layout(
    title="Concentration of Data Scientist Employment <br><sub> by Metropolitan Area, 2024",
    title_x=0.5,
    annotations=[
        dict(
            text="Source: Bureau of Labor Statistics - OEWS. <br>"
            "Note: Several MSA boundaries were redefined in 2024, "\
            "which may affect comparability of employment growth.<br>"
            "Data is not reported for the Raleigh-Cary MSA in 2021.",
            xref="paper", yref="paper",
            x=0, y=-0.15,
            showarrow=False,
            align="left",  # <-- aligns text to the left
            font=dict(size=10, color="gray")
        )
    ]
)

In [100]:
# Create the subplots
combo = make_subplots(
    rows=1, cols=2,
    column_widths=[0.7, 0.7],
    specs=[[{"type": "table"}, {"type": "mapbox"}]]
)

# Add the table trace
for trace in LQ_table.data:
    combo.add_trace(trace, row=1, col=1)

# Add the map trace
for trace in LQ_map.data:
    combo.add_trace(trace, row=1, col=2)

# Copy layout settings from the original map
combo.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=3,
    mapbox_center={"lat": 37.0902, "lon": -95.7129},
    coloraxis=dict(colorscale="Viridis", colorbar=dict(title="Concentration")),
    #height=600,
    #width=1000,
    title_text="Employment Concentration by Metropolitan Area",
    annotations=[
        dict(
            text="Source: Bureau of Labor Statistics - OEWS <BR>"
            "Note: Concentration = 1 is equal to the national average; Concentration > 1 higher than national average; Concentration <1 lower than national average",
            xref="paper", yref="paper",
            x=0, y=-0.1,
            showarrow=False,
            align="left",
            font=dict(size=10, color="gray")
        )
    ]
)


In [99]:
combo.write_html("map_LQ.html")

# Where are the fastest growing metros?

For this analysis we will only focus on the top 25 MSAs based on employment count. Some metros are growing really fast because they have such a small number of employees to begin with. Although understanding that type of growth is helpful, this analysis will focus on already established areas (top 25 metros).

Note: When comparing changes in BLS OEWS numbers over time, it's important to ensure consistency in job definitions and estimation methodology.
* The Standard Occupational Classification (SOC) system was updated in 2018, which reclassified many occupations. Therefore, data prior to 2018 may not be comparable to more recent years.
* In 2021, the OEWS program adopted the MB3 estimation methodology. Only estimates from 2021 onward, which use the MB3 model, should be compared for accurate trend analysis.

## Clean historical dfs

In [72]:
# read-in other data
Employment2021 = pd.read_excel('Data\\MSA_M2021_dl.xlsx', sheet_name="MSA_M2021_dl")  ### read in the data
Employment2022 = pd.read_excel('Data\\MSA_M2022_dl.xlsx', sheet_name="MSA_M2022_dl")  ### read in the data
Employment2023 = pd.read_excel('Data\\MSA_M2023_dl.xlsx', sheet_name="MSA_M2023_dl")  ### read in the data

In [73]:
MSA_employ2021 = create_MSAEmploy(Employment2021) ### Function defined in mapping section
MSA_employ2021 = MSA_employ2021[['AREA', 'TOT_EMP']] ### Area titles may be written differently, but the codes should be consistent overtime
MSA_employ2021 = MSA_employ2021.rename(columns={'TOT_EMP': 'TOT_EMP2021'})

MSA_employ2022 = create_MSAEmploy(Employment2022) 
MSA_employ2022 = MSA_employ2022[['AREA', 'TOT_EMP']]
MSA_employ2022 = MSA_employ2022.rename(columns={'TOT_EMP': 'TOT_EMP2022'})

MSA_employ2023 = create_MSAEmploy(Employment2023) 
MSA_employ2023 = MSA_employ2023[['AREA', 'TOT_EMP']]
MSA_employ2023 = MSA_employ2023.rename(columns={'TOT_EMP': 'TOT_EMP2023'})

In [74]:
# Pull the 2024 values defined in the map section
MSA_employ2024 = MSA_employ[['AREA', 'AREA_TITLE', 'TOT_EMP']]
MSA_employ2024 = MSA_employ2024.rename(columns={'TOT_EMP': 'TOT_EMP2024'})

## Map changes in MSA definitions
The 2024 MSA definitions were slightly altered based on what was used in the 2021-2023 period. One notable metro area that was reassigned is the Boston area. For the sake of comparison, we will still look at growth rates, but this is notable limitation to this analysis (mostly affecting the Northeast). 

2021 definitions: https://www.bls.gov/oes/2021/may/msa_def.htm

2024 definitions: https://www.bls.gov/oes/oes_doc.htm

In [75]:
# Df list
dfs = [MSA_employ2021, MSA_employ2022, MSA_employ2023]

# 2) Define a mapping old→new
area_map = {
    71650: 14460,  # Boston-Cambridge-Newton, MA-NH
    77200: 39300,  # Providence-Warwick, RI-MA
    17460: 17410,  # Cleveland, OH
    73450: 25540,  # Hartford-West Hartford-East Hartford, CT
    76750: 38860,  # Portland-South Portland, ME
}

# 3) Loop and replace
for df in dfs:
    df['AREA'].replace(area_map, inplace=True)


In [76]:
MSA_employ_trend = pd.merge(MSA_employ2024, MSA_employ2021, left_on='AREA', right_on='AREA', how='left') ### only merging matches with 2024
MSA_employ_trend = pd.merge(MSA_employ_trend, MSA_employ2022, left_on='AREA', right_on='AREA', how='left')   
MSA_employ_trend = pd.merge(MSA_employ_trend, MSA_employ2023, left_on='AREA', right_on='AREA', how='left')  

new_order = ['AREA', 'AREA_TITLE', 'TOT_EMP2021', 'TOT_EMP2022', 'TOT_EMP2023','TOT_EMP2024']
MSA_employ_trend = MSA_employ_trend[new_order]


In [77]:
MSA_employ_trend['PercentGrowth_2yr'] = ((MSA_employ_trend['TOT_EMP2023'] - MSA_employ_trend['TOT_EMP2021']) / MSA_employ_trend['TOT_EMP2021']) * 100
MSA_employ_trend = MSA_employ_trend.sort_values('PercentGrowth_2yr', ascending=False) ### Sort by fastest growing 

MSA_employ_trend = MSA_employ_trend[MSA_employ_trend['TOT_EMP2024']>=1000] ### Only want to look at metros with a large starting base

## Fastest growing visual

In [103]:
formatted_emp2021 = [f"{x:,.0f}" for x in MSA_employ_trend["TOT_EMP2021"]]
formatted_emp2022 = [f"{x:,.0f}" for x in MSA_employ_trend["TOT_EMP2022"]]
formatted_emp2023 = [f"{x:,.0f}" for x in MSA_employ_trend["TOT_EMP2023"]]
formatted_emp2024 = [f"{x:,.0f}" for x in MSA_employ_trend["TOT_EMP2024"]]
formatted_growth = [f"{x:.1f}%" for x in MSA_employ_trend["PercentGrowth_2yr"]]

fig = go.Figure(data=[go.Table(
    header=dict(values=['Metropolitan Area', '2021 Employee Count',  '2022 Employee Count', '2023 Employee Count', '2024 Employee Count', 'Percent Growth: 2021-2023'],
                fill_color='lightblue',
                align='left'),
    cells=dict(values=[
        MSA_employ_trend["AREA_TITLE"],
        formatted_emp2021,
        formatted_emp2022,
        formatted_emp2023,
        formatted_emp2024,
        formatted_growth
    ],
    fill_color='aliceblue',
    align='left'))
])

fig.update_layout(
    title="Metro Areas with Greater than 1,000 Data Science Employees<br><sup> Sorted by Employment Growth (2021–2023)",
    title_x=0.5,
    annotations=[
        dict(
            text="Source: Bureau of Labor Statistics-OEWS. <br>"\
                "Note: Metropolitan Statistical Area (MSA) boundaries were redefined in 2024,"\
                " which may limit comparability with prior years. This analysis focuses on employment growth <br>"\
                "from 2021 to 2023, when consistent MSA definitions were in use.",
            xref="paper", yref="paper",
            x=0, y=-0.13,
            showarrow=False,
            align="left",  # <-- aligns text to the left
            font=dict(size=10, color="gray")
        )
    ]
)


In [79]:
fig.write_html("Growth_table_interactive.html")

# Cost of living and salary

This analysis offers a high-level perspective on purchasing power across U.S. metropolitan areas by comparing median salaries for data scientists with local cost of living, as measured by the Regional Price Parity (RPP) Index.

There are some limitations to note. The data comes from two different sources with distinct methodologies: salary estimates are based on occupational surveys (2024), while RPP figures are derived from price level indices and expenditure weights (2023). These were the most recent data available at the time of analysis. Despite these differences, the combined view provides useful insight into how far a salary may go in different metro areas.


## Calculate Top 25

In [80]:
# Based on Employment Counts
top25_LQ = LQ_df.sort_values('LOC_QUOTIENT')
top25_LQ = LQ_df.nlargest(25, 'LOC_QUOTIENT')
top25_LQ['Rank Concentration']  = top25_LQ['LOC_QUOTIENT'].rank(method='min', ascending=False).astype(int)
top25_LQ = top25_LQ[['Rank Concentration', 'AREA_TITLE', 'LOC_QUOTIENT','TOT_EMP', 'AREA', 'A_MEDIAN']] ### Keeping 'AREA' & 'A_MEDIAN' for later on analysis
top25_LQ = top25_LQ.sort_values('TOT_EMP', ascending=False)


In [81]:
# Based on Employment Counts
top25 = LQ_df.copy()
top25["TOT_EMP"] = pd.to_numeric(LQ_df["TOT_EMP"], errors="coerce") ### Convert to numeric 
top25 = top25.nlargest(25, 'TOT_EMP')
top25['Rank Employees']  = top25['TOT_EMP'].rank(method='min', ascending=False).astype(int)
top25 = top25[['Rank Employees','AREA_TITLE', 'LOC_QUOTIENT','TOT_EMP', 'AREA', 'A_MEDIAN']]
top25 = top25.sort_values('TOT_EMP', ascending=False)

In [82]:
# Compare the top 25 
compare25 = pd.merge(top25_LQ, top25, left_on=('AREA_TITLE', 'LOC_QUOTIENT','TOT_EMP', 'AREA', 'A_MEDIAN'), 
                     right_on=('AREA_TITLE', 'LOC_QUOTIENT','TOT_EMP', 'AREA', 'A_MEDIAN'), how='outer')
compare25 = compare25.sort_values('TOT_EMP', ascending=False)

# Identifying if there is overlap in the dataset
def null_status(row):
    if pd.isnull(row['Rank Concentration']):
        return 'Employee Count'
    elif pd.isnull(row['Rank Employees']):
        return 'Concentration'
    else:
        return 'Both'

compare25['Overlap'] = compare25.apply(null_status, axis=1)


In [104]:
formatted_emp_ct= [f"{x:,.0f}" for x in compare25["TOT_EMP"]]

fig = go.Figure(data=[go.Table(
    header=dict(values=['Top 25','Metropolitan Area', 'Rank Employees', 'Employee Count', 'Rank Concentration', 'Concentration'],
                fill_color='#C04000',
                align='left', 
                font_color='white'),
    cells=dict(values=[
        compare25['Overlap'], 
        compare25['AREA_TITLE'], 
        compare25['Rank Employees'],
        formatted_emp_ct,
        compare25['Rank Concentration'],
        compare25['LOC_QUOTIENT']
    ],
    fill_color='#FAD5A5',
    align='left'))
])

fig.update_layout(
    title="Top 25 Metros by Employee Count vs. Concentration",
    title_x=0.5,
    annotations=[
        dict(
            text="Source: Bureau of Labor Statistics - OEWS. <br>"\
                "Note: 'null' means it was not in the top 25 for that column",
            xref="paper", yref="paper",
            x=0, y=-0.13,
            showarrow=False,
            align="left",  # <-- aligns text to the left
            font=dict(size=10, color="gray")
        )
    ]
)


In [84]:
fig.write_html("top25_comparison_table.html")

## Median Salary

### TOP 25 Employee Count

Choose to look at medians because the means are affected by outliers

In [105]:
# Read-in the BEA's Regional price parity
RPP = pd.read_excel('Data\\BEA_RPP.xlsx', sheet_name='Data')

In [106]:
# Top 25 metros based on total employee counts and employment concentration
top25_combo_wsalary = compare25.sort_values('TOT_EMP')
top25_combo_wsalary = top25_combo_wsalary[['AREA', 'AREA_TITLE', 'TOT_EMP', 'A_MEDIAN', 'Overlap', 'LOC_QUOTIENT']]

RPP_Salary_Combo = pd.merge(RPP, top25_combo_wsalary, left_on='GeoFips', right_on='AREA', how='right')  ### Only want RPP codes that match the selected metros
RPP_Salary_Combo =RPP_Salary_Combo[['AREA_TITLE', 'RPP', 'A_MEDIAN', 'TOT_EMP', 'Overlap', 'LOC_QUOTIENT']]

In [108]:
# Add a formatted employment column
RPP_Salary_Combo["Formatted_EMP"] = RPP_Salary_Combo["TOT_EMP"].apply(lambda x: f"{x:,.0f}")

fig = px.scatter(
    RPP_Salary_Combo,
    x='RPP',
    y='A_MEDIAN',
    hover_name='AREA_TITLE',
    color='Overlap',  
    title='Median Data Scientist Salary vs. Cost of Living <br><sup> Top 25 Metros Based on Employee Count and Concentration',
    labels={
        'RPP': 'Regional Price Parity Index (2023)',
        'A_MEDIAN': 'Annual Median Salary ($ in 2024)',
        'Formatted_EMP': 'Employee Count',
        'LOC_QUOTIENT': "Concentration"
    },
    color_discrete_map={
        'Employee Count': '#FF8C00',   # dark orange
        'Concentration': '#A7C7E7',    # light blue
        'Both': '#002147'              # deep navy
    },
    hover_data={
        'RPP': True,
        'A_MEDIAN': True,
        'Overlap': False,
        'Formatted_EMP': True,
        'LOC_QUOTIENT': True
    },
)

fig.update_traces(
    marker=dict(size=12, line=dict(width=1, color='DarkSlateGrey'))
)

fig.update_layout(
    title_font_size=20,
    xaxis=dict(tickformat=".1f"),
    yaxis=dict(tickprefix="$"),
    hovermode='closest',
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend_title_text=None,
    shapes=[
        dict(type='line', x0=100, x1=100, y0=RPP_Salary_Combo['A_MEDIAN'].min(), y1=RPP_Salary_Combo['A_MEDIAN'].max(),
             line=dict(color='#bf0000', width=2, dash='dash')),
        dict(type='line', x0=RPP_Salary_Combo['RPP'].min(), x1=RPP_Salary_Combo['RPP'].max(), y0=112590, y1=112590,
             line=dict(color='#bf0000', width=2, dash='dash'))
    ],
    annotations=[
        dict(
            text="Source: Annual Median Salary - BLS OEWS, Cost of Living Index - BEA Regional Price Parity.",
            xref="paper", yref="paper",
            x=0, y=-0.17,
            showarrow=False,
            font=dict(size=10, color="gray")
        ),
        dict(
            x=100, y=1.02, xref='x', yref='paper',
            text='National Average: Cost of Living = 100',
            showarrow=False,
            font=dict(color='#bf0000'),
            align='center'
        ),
        dict(
            x=0.99, y=109500, xref='paper', yref='y',
            text='National Median Data Scientist Salary: $112,590',
            showarrow=False,
            font=dict(color='#bf0000'),
            align='left'
        )
    ]
)

fig.show()


In [90]:
fig.write_html("top25Comparison_costofliving_Median.html")